# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it  
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# create new csv file to collect all data together
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    # write column header
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        # do not write if artist data is missing
        if (row[0] == ''):
            continue
        # write data
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
    

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a keyspace named 'jhkim'
query = "CREATE KEYSPACE IF NOT EXISTS jhkim\
         WITH REPLICATION =\
         {'class': 'SimpleStrategy', 'replication_factor': 1}"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('jhkim')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




- Database Schema Diagram  
  `song_play`  
  
  | column | datatype | primary key |
  |:---: | :---:| :---:|
  |session_id| int|partition key|
  |item_in_session| int|clustering key|
  |artist| text||
  |song_title| text||
  |song_length| float||

In [8]:
# columns:
# For this query 1, 
# it seems to need only 
# session_id, item_in_session, artist, song_title, song_length.

# primary key:
# Following the order of columns came across in where clause,
# session_id is a partition key and
# item_in_session is a clustering column.

query = "CREATE TABLE IF NOT EXISTS song_play\
        (session_id varint,\
        item_in_session varint,\
        artist text,\
        song_title text,\
        song_length float,\
        PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Add data into table
        query = "INSERT INTO song_play (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
    
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Execute SELECT statement to verify the data was entered into the table
select = "SELECT artist, song_title, song_length FROM song_play\
          WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(select)
except Exception as e:
    print(e)
else:
    # convert resultset to dataframe
    df = pd.DataFrame(rows._current_rows)

In [11]:
df

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

- Database Schema Diagram  
  `user_playhistory`  
  | column | datatype | primary key |
  |:---: | :---:| :---:|
  |user_id| int|partition key 1|
  |session_id| int|partition key 2|
  |item_in_session| int|clustering key|
  |artist| text||
  |song_title| text||
  |first_name| text||
  |last_name| text||

In [12]:
# columns:
# For this query 2, 
# it seems to need only 
# user_id, session_id, item_in_session, artist, song_title, first_name, last_name.

# primary key:
# Following the order of columns came across in where clause,
# user_id and session_id is a partition key.
# And item_in_session is added as a clustering column,
# because the result should be sorted by item_in_session.

query = "CREATE TABLE IF NOT EXISTS user_playhistory\
        (user_id varint,\
        session_id varint,\
        item_in_session varint,\
        artist text,\
        song_title text,\
        first_name text,\
        last_name text,\
        PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Add data into table
        query = "INSERT INTO user_playhistory (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
# SELECT statement to verify the data was entered into the table
select = "SELECT artist, song_title, first_name, last_name FROM user_playhistory\
          WHERE user_id = 10 and session_id = 182"

try:
    rows = session.execute(select)
except Exception as e:
    print(e)
else:
    # convert resultset to dataframe
    df = pd.DataFrame(rows._current_rows)

In [15]:
df

,artist,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


- Database Schema Diagram  
`song_userlist`  
  | column | datatype | primary key |
  |:---: | :---:| :---:|
  |song_title| text|partition key|
  |artist| text|clustering key|
  |first_name| text||
  |last_name| text||

In [16]:
# columns:
# For this query 3, 
# it seems to need only 
# song_title, first_name, last_name.
# But I added artist as a clustering column in order to make primary key unique.

# primary key:
# Following the column came across in where clause,
# song_title is a partition key and
# artist is a clustering column.

query = "CREATE TABLE IF NOT EXISTS song_userlist\
        (song_title text,\
        artist text,\
        first_name text,\
        last_name text,\
        PRIMARY KEY(song_title, artist))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Add data into table
        query = "INSERT INTO song_userlist (song_title, artist, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
    
        session.execute(query, (line[9], line[0], line[1], line[4]))

In [18]:
# SELECT statement to verify the data was entered into the table
select = "SELECT first_name, last_name FROM song_userlist\
          WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(select)
except Exception as e:
    print(e)
else:
    # convert resultset to dataframe
    df = pd.DataFrame(rows._current_rows)

In [19]:
df

,first_name,last_name
0,Sara,Johnson


### Drop the tables before closing out the sessions

In [20]:
# Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS song_play"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_playhistory"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_userlist"

try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()